In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

/tmp/ipykernel_2870663/1226147920.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import sys
import os
import copy
import pathlib
from pathlib import Path
import time
import gc

In [3]:
import torch
import numpy as np
import sklearn
from tqdm import tqdm, trange

In [4]:
from tqdm import tqdm, trange
# from tqdm import trange

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

import torch
import torchvision
import torchvision.transforms as transforms

# import cuml

# for creating validation set
import sklearn
from sklearn.model_selection import train_test_split

%matplotlib inline
%matplotlib inline
# %matplotlib inline
# %matplotlib qt

In [5]:
dir_save = '/media/rich/bigSSD/analysis_data/ROI_net_training/testing_dispatcher_20220504'

In [6]:
dir_github = '/media/rich/Home_Linux_partition/github_repos'

In [7]:
path_data_training = '/media/rich/bigSSD/for_Josh/SimCLR-Label-Data/masks_20211202_balanced.h5'

path_dataClassification_stat = '/media/rich/bigSSD/for_Josh/SimCLR-Label-Data/mouse2_6__20210409/stat.npy'
path_dataClassification_labels = '/media/rich/bigSSD/for_Josh/SimCLR-Label-Data/mouse2_6__20210409/labels_round2_sesh2.npy'

In [8]:
import sys

sys.path.append(dir_github)
# dir_folders = f'{base_dir}/label_data'

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import torch_helpers, math_functions, classification, h5_handling, plotting_helpers, indexing, misc, decomposition
from GCaMP_ROI_classifier import util, models, training, augmentation, dataset

In [9]:
def write_to_log(path_log, text, mode='a', start_on_new_line=True):
    with open(path_log, mode=mode) as log:
        if start_on_new_line==True:
            log.write('\n')
        log.write(text)

# Parse arguments

In [10]:
params = {
    'useGPU_training': True,
    'pre_head_fc_sizes': [1024, 512],
    'post_head_fc_sizes': [64],
    'block_to_freeze': 7,
    'useGPU_dataloader': False,
    'dataloader_kwargs':{
        'batch_size': 1024,
        'shuffle': True,
        'drop_last': True,
        'pin_memory': True,
        'num_workers': 36,
        'persistent_workers': True,
        'prefetch_factor': 3,
    },
    'lr': 1*10**-4,
    'gamma': 1-0.0000,
    'n_epochs': 9999999,
    'temperature': 0.5,
}

dir_save = Path(r'/media/rich/bigSSD/')
fileName_model = 'EfficientNet_b0_7unfrozen_simCLR'
path_saveModel = f'{dir_save / fileName_model}.pth'
path_saveLog = dir_save / 'log.txt'
path_saveLoss = dir_save / 'loss.npy'

pref_saveModelIteratively = True
pref_saveLogs = True

device_train = torch_helpers.set_device(use_GPU=params['useGPU_training'])

device: 'cuda'


## Import unlabeled training data

In [11]:
import scipy.sparse

sf_sparse = scipy.sparse.load_npz('/media/rich/bigSSD/analysis_data/ROIs_for_training/sf_sparse_36x36_20220503.npz')

In [12]:
sf_dense = torch.as_tensor(sf_sparse.toarray().reshape(sf_sparse.shape[0], 36,36), dtype=torch.float32)

toss any NaNs

In [13]:
print(f'Number of masks: {sf_dense.shape}')

ROIs_without_NaNs = ~torch.any(torch.any(torch.isnan(sf_dense), dim=1), dim=1)
ROIs_nonAllZero = (torch.max(torch.max(sf_dense, dim=1)[0], dim=1)[0] > 0)
ROIs_toKeep = torch.where(ROIs_without_NaNs * ROIs_nonAllZero)
masks_cat = sf_dense[ROIs_toKeep]

print(f'Number of masks: {masks_cat.shape}')

Number of masks: torch.Size([3094029, 36, 36])
Number of masks: torch.Size([3094025, 36, 36])


# Train

### Define New model = model + pre-head + latent layer OR classification layer

In [14]:
class ModelTackOn(torch.nn.Module):
    def __init__(self, base_model, un_modified_model, pre_head_fc_sizes=[100], post_head_fc_sizes=[100], classifier_fc_sizes=None):
            super(ModelTackOn, self).__init__()
            self.base_model = base_model
            final_base_layer = list(un_modified_model.children())[-1]
            # final_base_layer = list(list(model.children())[-1].children())[-1]
            # print(final_base_layer)

            self.pre_head_fc_lst = []
            self.post_head_fc_lst = []
            self.classifier_fc_lst = []

            self.init_prehead(final_base_layer, pre_head_fc_sizes)
            self.init_posthead(pre_head_fc_sizes[-1], post_head_fc_sizes)
            if classifier_fc_sizes is not None:
                self.init_classifier(pre_head_fc_sizes[-1], classifier_fc_sizes)
    
    def init_prehead(self, prv_layer, pre_head_fc_sizes):
        for i, pre_head_fc in enumerate(pre_head_fc_sizes):
            if i == 0:
                in_features = prv_layer.in_features if hasattr(prv_layer,'in_features') else 1280
            else:
                in_features = pre_head_fc_sizes[i - 1]
            fc_layer = torch.nn.Linear(in_features=in_features, out_features=pre_head_fc)
            self.add_module(f'PreHead_{i}', fc_layer)
            self.pre_head_fc_lst.append(fc_layer)

#             if i < len(pre_head_fc_sizes) - 1:
            non_linearity = torch.nn.ReLU()
            self.add_module(f'PreHead_{i}_NonLinearity', non_linearity)
            self.pre_head_fc_lst.append(non_linearity)

    def init_posthead(self, prv_size, post_head_fc_sizes):
        for i, post_head_fc in enumerate(post_head_fc_sizes):
            if i == 0:
                in_features = prv_size
            else:
                in_features = post_head_fc_sizes[i - 1]
            fc_layer = torch.nn.Linear(in_features=in_features, out_features=post_head_fc)
            self.add_module(f'PostHead_{i}', fc_layer)
            self.post_head_fc_lst.append(fc_layer)

            if i < len(post_head_fc_sizes) - 1:
                non_linearity = torch.nn.ReLU()
                self.add_module(f'PostHead_{i}_NonLinearity', non_linearity)
                self.pre_head_fc_lst.append(non_linearity)
    
    def init_classifier(self, prv_size, classifier_fc_sizes):
            for i, classifier_fc in enumerate(classifier_fc_sizes):
                if i == 0:
                    in_features = prv_size
                else:
                    in_features = classifier_fc_sizes[i - 1]
            fc_layer = torch.nn.Linear(in_features=in_features, out_features=classifier_fc)
            self.add_module(f'Classifier_{i}', fc_layer)
            self.classifier_fc_lst.append(fc_layer)

    def reinit_classifier(self):
        for i_layer, layer in enumerate(self.classifier_fc_lst):
            layer.reset_parameters()
    
#     def forward(self, X):
#         interim = self.base_model(X)
#         interim = self.get_head(interim)
#         interim = self.get_latent(interim)
#         return interim

    def forward_classifier(self, X):
        interim = self.base_model(X)
        interim = self.get_head(interim)
        interim = self.classify(interim)
        return interim

    def forward_latent(self, X):
        interim = self.base_model(X)
        interim = self.get_head(interim)
        interim = self.get_latent(interim)
        return interim


    def get_head(self, base_out):
        # print('base_out', base_out.shape)
        head = base_out
        for pre_head_layer in self.pre_head_fc_lst:
          # print('pre_head_layer', pre_head_layer.in_features)
          head = pre_head_layer(head)
          # print('head', head.shape)
        return head

    def get_latent(self, head):
        latent = head
        for post_head_layer in self.post_head_fc_lst:
            latent = post_head_layer(latent)
        return latent

    def classify(self, head):
        logit = head
        for classifier_layer in self.classifier_fc_lst:
            logit = classifier_layer(logit)
        return logit

    def set_pre_head_grad(self, requires_grad=True):
        for layer in self.pre_head_fc_lst:
            for param in layer.parameters():
                param.requires_grad = requires_grad
                
    def set_post_head_grad(self, requires_grad=True):
        for layer in self.post_head_fc_lst:
            for param in layer.parameters():
                param.requires_grad = requires_grad

    def set_classifier_grad(self, requires_grad=True):
        for layer in self.classifier_fc_lst:
            for param in layer.parameters():
                param.requires_grad = requires_grad

    def prep_contrast(self):
        self.set_pre_head_grad(requires_grad=True)
        self.set_post_head_grad(requires_grad=True)
        self.set_classifier_grad(requires_grad=False)

    def prep_classifier(self):
        self.set_pre_head_grad(requires_grad=False)
        self.set_post_head_grad(requires_grad=False)
        self.set_classifier_grad(requires_grad=True)


In [15]:
# import torchvision.models

# # base_model = torchvision.models.resnet101(pretrained=True)
# base_model = torchvision.models.resnet18(pretrained=True)

# for param in base_model.parameters():
#     param.requires_grad = False

# retrain = list(base_model.children())[-1:]
# for layer in retrain:
#     params = layer.parameters()
#     for param in params:
#         param.requires_grad = True

In [16]:
import torchvision.models

# base_model_frozen = torchvision.models.resnet101(pretrained=True)
# base_model_frozen = torchvision.models.resnet18(pretrained=True)
# base_model_frozen = torchvision.models.wide_resnet50_2(pretrained=True)
# base_model_frozen = torchvision.models.resnet50(pretrained=True)

base_model_frozen = torchvision.models.efficientnet_b0(pretrained=True)


for param in base_model_frozen.parameters():
    param.requires_grad = False

start with a pretrained resnet model, and chop off the final layer. This will be used as the base on which we add the pre-head layers (for expressivity), latent layers (for simCLR), or classification layers (for post-hoc logistic regression)

In [17]:
model_chopped = torch.nn.Sequential(*(list(base_model_frozen.children())[:-1] + [torch.nn.Flatten()]))

### Make combined model
'model' has two forward methods. One for generating latents (for simCLR) and one for classifying labels

In [18]:
model = ModelTackOn(
    model_chopped,
    base_model_frozen,
    pre_head_fc_sizes=params['pre_head_fc_sizes'], 
    post_head_fc_sizes=params['post_head_fc_sizes'], 
    classifier_fc_sizes=None
)
model.train();

In [19]:
# unfreeze particular blocks in CNN model

for name, param in list(model.named_parameters()):
    # print(name)
    if name[:10] == 'base_model':
        if int(name[13]) < params['block_to_freeze']:
            param.requires_grad = False
        elif int(name[13]) >= params['block_to_freeze']:
            param.requires_grad = True

for name, param in list(model.named_parameters()):
    if param.requires_grad:
        print(name)

base_model.0.7.0.block.0.0.weight
base_model.0.7.0.block.0.1.weight
base_model.0.7.0.block.0.1.bias
base_model.0.7.0.block.1.0.weight
base_model.0.7.0.block.1.1.weight
base_model.0.7.0.block.1.1.bias
base_model.0.7.0.block.2.fc1.weight
base_model.0.7.0.block.2.fc1.bias
base_model.0.7.0.block.2.fc2.weight
base_model.0.7.0.block.2.fc2.bias
base_model.0.7.0.block.3.0.weight
base_model.0.7.0.block.3.1.weight
base_model.0.7.0.block.3.1.bias
base_model.0.8.0.weight
base_model.0.8.1.weight
base_model.0.8.1.bias
PreHead_0.weight
PreHead_0.bias
PreHead_1.weight
PreHead_1.bias
PostHead_0.weight
PostHead_0.bias


## Define augmentation pipeline

In [20]:
import torchvision.transforms    

transforms = torch.nn.Sequential(
    augmentation.Scale_image_sum(sum_val=1, epsilon=1e-9, min_sub=True),
    
    augmentation.AddPoissonNoise(  
        #scaler_bounds=(10**(4.5), 10**(6.)),
        scaler_bounds=(10**(4), 10**(5)),
        prob=0.5,
        base=1000,
        scaling='log'),
    
    augmentation.Horizontal_stripe_scale(alpha_min_max=(0.5, 1), im_size=(36,36), prob=0.5),
    augmentation.Horizontal_stripe_shift(alpha_min_max=(1,3), im_size=(36,36), prob=0.5),

    torchvision.transforms.RandomHorizontalFlip(p=0.5),  
    torchvision.transforms.RandomAffine(
        degrees=(-180,180),
        translate=(0.1, 0.1), #0, .3, .45 (DEFAULT)
        scale=(0.6, 1.2), # no scale (1,1), (0.4, 1.5)
        shear=(-15, 15, -15, 15),
        interpolation=torchvision.transforms.InterpolationMode.BILINEAR, 
        fill=0, 
        fillcolor=None, 
        resample=None),

    augmentation.AddGaussianNoise(  
        mean=0, 
        std=0.0010,
        prob=0.5),
    
    augmentation.ScaleDynamicRange(scaler_bounds=(0,1), epsilon=1e-9), # just clamping, both this and clamping = normalizing (DEFAULT)
    
    augmentation.WarpPoints(
        r=[0.3, 0.6],
        cx=[-0.3, 0.3],
        cy=[-0.3, 0.3], 
        dx=[-0.24, 0.24], 
        dy=[-0.24, 0.24], 
        n_warps=2,
        prob=0.5,
        img_size_in=[36, 36],
#         img_size_out=[224, 224],
        img_size_out=[180, 180],
#         img_size_out=[36, 36],
                           ),
    
#     augmentation.Check_NaN(),
    
    augmentation.TileChannels(dim=0, n_channels=3),
# #     torchvision.transforms.Resize(size=(180,180), 
# #                                   interpolation=torchvision.transforms.InterpolationMode.BILINEAR), # To do or not to do (DEFAULT)
    
)
    

scripted_transforms = torch.jit.script(transforms)

In [21]:
device_dataloader = torch_helpers.set_device(use_GPU=params['useGPU_dataloader'])

dataset_train = dataset.dataset_simCLR(
    torch.as_tensor(masks_cat, device=device_dataloader, dtype=torch.float32), 
    torch.as_tensor(torch.zeros(masks_cat.shape[0]), device=device_dataloader, dtype=torch.float32),

    n_transforms=2,
    class_weights=np.array([1]),
    # class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
    transform=scripted_transforms,
    # DEVICE='cpu',
    DEVICE=device_dataloader,
    dtype_X=torch.float32,
    dtype_y=torch.int64,
    # temp_uncertainty=1
)

dataloader_train = torch.utils.data.DataLoader(
    dataset_train,
    **params['dataloader_kwargs']

#     batch_size=1024,
#     shuffle=True,
#     drop_last=True,
#     pin_memory=True,
#     num_workers=36,
#     persistent_workers=True,
#     prefetch_factor=3,
    
#     batch_size=1024,
#     shuffle=False,
#     drop_last=True,
#     pin_memory=False,
#     num_workers=36,
#     persistent_workers=True,
#     prefetch_factor=3,
)

device: 'cpu'


In [22]:
# %matplotlib notebook

# idx_rand = np.random.randint(0,masks_cat2.shape[0], 10)
# for ii in idx_rand:
#     fig, axs = plt.subplots(1,2)
#     # print(dataset_train[ii][0][0][0].shape)
#     axs[0].imshow(dataset_train[ii][0][0][0].cpu())
#     axs[1].imshow(dataset_train[ii][0][1][0].cpu())

## Training

In [23]:
model.to(device_train)
model.prep_contrast()
model.forward = model.forward_latent

In [24]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

criterion = [CrossEntropyLoss()]
optimizer = Adam(
    model.parameters(), 
    lr=params['lr'],
#     lr=1*10**-4,
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,
                                                   gamma=params['gamma'],
#                                                    gamma=1,
                                                  )

criterion = [_.to(device_train) for _ in criterion]

In [ ]:
losses_train, losses_val = [], [np.nan]
for epoch in tqdm(range(params['n_epochs'])):
    print(f'epoch: {epoch}')
    
    losses_train = training.epoch_step(
        dataloader_train, 
        model, 
        optimizer, 
        criterion,
        scheduler=scheduler,
        temperature=params['temperature'],
        # l2_alpha,
        mode='semi-supervised',
        loss_rolling_train=losses_train, 
        loss_rolling_val=losses_val,
        device=device_train, 
        verbose=2,
        verbose_update_period=1,

#                                     do_validation=False,
#                                     X_val=x_feed_through_val,
#                                     y_val=torch.as_tensor(y_val, device=DEVICE)
)
    
    ## save loss stuff
    if pref_saveLogs:
        write_to_log(path_log=path_saveLog, text=f'time:{time.ctime()}, completed epoch: {epoch}, loss: {losses_train[-1]}, lr: {scheduler.get_last_lr()[0]}')
        np.save(path_saveLoss, losses_train)
    
    ## if loss becomes NaNs, don't save the network and stop training
    if torch.isnan(losses_train[-1]):
        break
        
    ## save model
    if pref_saveModelIteratively:
        torch.save(model.state_dict(), path_saveModel)

  0%|                                               | 0/9999999 [00:00<?, ?it/s]

epoch: 0
Iter: 0/3021, loss_train: 7.6008, loss_val: nan, pos_over_neg: 1.021756887435913 lr: 0.0001
Iter: 1/3021, loss_train: 7.565, loss_val: nan, pos_over_neg: 1.0537643432617188 lr: 0.0001
Iter: 2/3021, loss_train: 7.4886, loss_val: nan, pos_over_neg: 1.1408154964447021 lr: 0.0001
Iter: 3/3021, loss_train: 7.4124, loss_val: nan, pos_over_neg: 1.2915561199188232 lr: 0.0001
Iter: 4/3021, loss_train: 7.3437, loss_val: nan, pos_over_neg: 1.5708016157150269 lr: 0.0001
Iter: 5/3021, loss_train: 7.2783, loss_val: nan, pos_over_neg: 2.1402266025543213 lr: 0.0001
Iter: 6/3021, loss_train: 7.2274, loss_val: nan, pos_over_neg: 2.954249382019043 lr: 0.0001
Iter: 7/3021, loss_train: 7.1915, loss_val: nan, pos_over_neg: 3.8929965496063232 lr: 0.0001
Iter: 8/3021, loss_train: 7.249, loss_val: nan, pos_over_neg: 4.432919502258301 lr: 0.0001
Iter: 9/3021, loss_train: 7.2081, loss_val: nan, pos_over_neg: 5.018045902252197 lr: 0.0001
Iter: 10/3021, loss_train: 7.1698, loss_val: nan, pos_over_neg: 4.2

Iter: 88/3021, loss_train: 6.8354, loss_val: nan, pos_over_neg: 43.760162353515625 lr: 0.0001
Iter: 89/3021, loss_train: 6.8146, loss_val: nan, pos_over_neg: 48.851261138916016 lr: 0.0001
Iter: 90/3021, loss_train: 6.8093, loss_val: nan, pos_over_neg: 41.814632415771484 lr: 0.0001
Iter: 91/3021, loss_train: 6.8204, loss_val: nan, pos_over_neg: 43.28696060180664 lr: 0.0001
Iter: 92/3021, loss_train: 6.7745, loss_val: nan, pos_over_neg: 56.589866638183594 lr: 0.0001
Iter: 93/3021, loss_train: 6.7896, loss_val: nan, pos_over_neg: 40.6494026184082 lr: 0.0001
Iter: 94/3021, loss_train: 6.7897, loss_val: nan, pos_over_neg: 43.2087287902832 lr: 0.0001
Iter: 95/3021, loss_train: 6.771, loss_val: nan, pos_over_neg: 48.807743072509766 lr: 0.0001
Iter: 96/3021, loss_train: 6.78, loss_val: nan, pos_over_neg: 53.1165657043457 lr: 0.0001
Iter: 97/3021, loss_train: 6.8065, loss_val: nan, pos_over_neg: 51.976200103759766 lr: 0.0001
Iter: 98/3021, loss_train: 6.7815, loss_val: nan, pos_over_neg: 71.778

Iter: 176/3021, loss_train: 6.7071, loss_val: nan, pos_over_neg: 84.76749420166016 lr: 0.0001
Iter: 177/3021, loss_train: 6.7194, loss_val: nan, pos_over_neg: 70.1619644165039 lr: 0.0001
Iter: 178/3021, loss_train: 6.7313, loss_val: nan, pos_over_neg: 50.301334381103516 lr: 0.0001
Iter: 179/3021, loss_train: 6.715, loss_val: nan, pos_over_neg: 59.340797424316406 lr: 0.0001
Iter: 180/3021, loss_train: 6.7239, loss_val: nan, pos_over_neg: 47.43552017211914 lr: 0.0001
Iter: 181/3021, loss_train: 6.7145, loss_val: nan, pos_over_neg: 53.7491340637207 lr: 0.0001
Iter: 182/3021, loss_train: 6.7351, loss_val: nan, pos_over_neg: 57.259178161621094 lr: 0.0001
Iter: 183/3021, loss_train: 6.7138, loss_val: nan, pos_over_neg: 57.80210876464844 lr: 0.0001
Iter: 184/3021, loss_train: 6.6973, loss_val: nan, pos_over_neg: 86.44732666015625 lr: 0.0001
Iter: 185/3021, loss_train: 6.7185, loss_val: nan, pos_over_neg: 71.84893035888672 lr: 0.0001
Iter: 186/3021, loss_train: 6.7022, loss_val: nan, pos_over_

Iter: 264/3021, loss_train: 6.7164, loss_val: nan, pos_over_neg: 56.449798583984375 lr: 0.0001
Iter: 265/3021, loss_train: 6.71, loss_val: nan, pos_over_neg: 69.50334167480469 lr: 0.0001
Iter: 266/3021, loss_train: 6.6901, loss_val: nan, pos_over_neg: 108.88590240478516 lr: 0.0001
Iter: 267/3021, loss_train: 6.675, loss_val: nan, pos_over_neg: 110.29940795898438 lr: 0.0001
Iter: 268/3021, loss_train: 6.6815, loss_val: nan, pos_over_neg: 136.855712890625 lr: 0.0001
Iter: 269/3021, loss_train: 6.6864, loss_val: nan, pos_over_neg: 126.91331481933594 lr: 0.0001
Iter: 270/3021, loss_train: 6.6948, loss_val: nan, pos_over_neg: 102.88433837890625 lr: 0.0001
Iter: 271/3021, loss_train: 6.682, loss_val: nan, pos_over_neg: 117.58699035644531 lr: 0.0001
Iter: 272/3021, loss_train: 6.6672, loss_val: nan, pos_over_neg: 123.74059295654297 lr: 0.0001
Iter: 273/3021, loss_train: 6.6968, loss_val: nan, pos_over_neg: 97.98333740234375 lr: 0.0001
Iter: 274/3021, loss_train: 6.6962, loss_val: nan, pos_ove

...

Iter: 1049/3021, loss_train: 6.5706, loss_val: nan, pos_over_neg: 91.85337829589844 lr: 0.0001
Iter: 1050/3021, loss_train: 6.5687, loss_val: nan, pos_over_neg: 79.33258819580078 lr: 0.0001
Iter: 1051/3021, loss_train: 6.5701, loss_val: nan, pos_over_neg: 98.234130859375 lr: 0.0001
Iter: 1052/3021, loss_train: 6.5457, loss_val: nan, pos_over_neg: 135.73501586914062 lr: 0.0001
Iter: 1053/3021, loss_train: 6.6161, loss_val: nan, pos_over_neg: 97.24614715576172 lr: 0.0001
Iter: 1054/3021, loss_train: 6.5773, loss_val: nan, pos_over_neg: 144.2130584716797 lr: 0.0001
Iter: 1055/3021, loss_train: 6.5936, loss_val: nan, pos_over_neg: 140.12806701660156 lr: 0.0001
Iter: 1056/3021, loss_train: 6.5473, loss_val: nan, pos_over_neg: 139.00344848632812 lr: 0.0001
Iter: 1057/3021, loss_train: 6.5726, loss_val: nan, pos_over_neg: 176.90676879882812 lr: 0.0001
Iter: 1058/3021, loss_train: 6.5625, loss_val: nan, pos_over_neg: 96.17406463623047 lr: 0.0001
Iter: 1059/3021, loss_train: 6.554, loss_val: na

Iter: 1136/3021, loss_train: 6.5394, loss_val: nan, pos_over_neg: 119.95569610595703 lr: 0.0001
Iter: 1137/3021, loss_train: 6.5649, loss_val: nan, pos_over_neg: 90.5282974243164 lr: 0.0001
Iter: 1138/3021, loss_train: 6.5355, loss_val: nan, pos_over_neg: 106.25878143310547 lr: 0.0001
Iter: 1139/3021, loss_train: 6.576, loss_val: nan, pos_over_neg: 78.8614730834961 lr: 0.0001
Iter: 1140/3021, loss_train: 6.5382, loss_val: nan, pos_over_neg: 123.21450805664062 lr: 0.0001
Iter: 1141/3021, loss_train: 6.5917, loss_val: nan, pos_over_neg: 63.884925842285156 lr: 0.0001
Iter: 1142/3021, loss_train: 6.5079, loss_val: nan, pos_over_neg: 123.59380340576172 lr: 0.0001
Iter: 1143/3021, loss_train: 6.5641, loss_val: nan, pos_over_neg: 85.61296081542969 lr: 0.0001
Iter: 1144/3021, loss_train: 6.5655, loss_val: nan, pos_over_neg: 88.60919189453125 lr: 0.0001
Iter: 1145/3021, loss_train: 6.555, loss_val: nan, pos_over_neg: 111.32308197021484 lr: 0.0001
Iter: 1146/3021, loss_train: 6.5671, loss_val: n

Iter: 1222/3021, loss_train: 6.5461, loss_val: nan, pos_over_neg: 149.07638549804688 lr: 0.0001
Iter: 1223/3021, loss_train: 6.5691, loss_val: nan, pos_over_neg: 113.75347137451172 lr: 0.0001
Iter: 1224/3021, loss_train: 6.5747, loss_val: nan, pos_over_neg: 92.91566467285156 lr: 0.0001
Iter: 1225/3021, loss_train: 6.545, loss_val: nan, pos_over_neg: 137.8286590576172 lr: 0.0001
Iter: 1226/3021, loss_train: 6.5504, loss_val: nan, pos_over_neg: 119.91869354248047 lr: 0.0001
Iter: 1227/3021, loss_train: 6.5187, loss_val: nan, pos_over_neg: 156.17721557617188 lr: 0.0001
Iter: 1228/3021, loss_train: 6.5433, loss_val: nan, pos_over_neg: 119.5831298828125 lr: 0.0001
Iter: 1229/3021, loss_train: 6.5768, loss_val: nan, pos_over_neg: 124.85914611816406 lr: 0.0001
Iter: 1230/3021, loss_train: 6.5523, loss_val: nan, pos_over_neg: 125.78163146972656 lr: 0.0001
Iter: 1231/3021, loss_train: 6.5676, loss_val: nan, pos_over_neg: 123.45030212402344 lr: 0.0001
Iter: 1232/3021, loss_train: 6.5727, loss_va

Iter: 1309/3021, loss_train: 6.5382, loss_val: nan, pos_over_neg: 113.45020294189453 lr: 0.0001
Iter: 1310/3021, loss_train: 6.5559, loss_val: nan, pos_over_neg: 66.12593078613281 lr: 0.0001
Iter: 1311/3021, loss_train: 6.572, loss_val: nan, pos_over_neg: 96.0416488647461 lr: 0.0001
Iter: 1312/3021, loss_train: 6.5592, loss_val: nan, pos_over_neg: 83.86682891845703 lr: 0.0001
Iter: 1313/3021, loss_train: 6.5365, loss_val: nan, pos_over_neg: 155.9613800048828 lr: 0.0001
Iter: 1314/3021, loss_train: 6.5728, loss_val: nan, pos_over_neg: 109.65827941894531 lr: 0.0001
Iter: 1315/3021, loss_train: 6.5414, loss_val: nan, pos_over_neg: 285.1993408203125 lr: 0.0001
Iter: 1316/3021, loss_train: 6.5163, loss_val: nan, pos_over_neg: 296.63385009765625 lr: 0.0001
Iter: 1317/3021, loss_train: 6.5626, loss_val: nan, pos_over_neg: 122.68668365478516 lr: 0.0001
Iter: 1318/3021, loss_train: 6.543, loss_val: nan, pos_over_neg: 144.9898223876953 lr: 0.0001
Iter: 1319/3021, loss_train: 6.5338, loss_val: na

Iter: 1396/3021, loss_train: 6.5374, loss_val: nan, pos_over_neg: 125.47113800048828 lr: 0.0001
Iter: 1397/3021, loss_train: 6.5522, loss_val: nan, pos_over_neg: 86.76875305175781 lr: 0.0001
Iter: 1398/3021, loss_train: 6.504, loss_val: nan, pos_over_neg: 84.27010345458984 lr: 0.0001
Iter: 1399/3021, loss_train: 6.5444, loss_val: nan, pos_over_neg: 93.54882049560547 lr: 0.0001
Iter: 1400/3021, loss_train: 6.5394, loss_val: nan, pos_over_neg: 132.93801879882812 lr: 0.0001
Iter: 1401/3021, loss_train: 6.5526, loss_val: nan, pos_over_neg: 97.32282257080078 lr: 0.0001
Iter: 1402/3021, loss_train: 6.5269, loss_val: nan, pos_over_neg: 146.22781372070312 lr: 0.0001
Iter: 1403/3021, loss_train: 6.5676, loss_val: nan, pos_over_neg: 112.19435119628906 lr: 0.0001
Iter: 1404/3021, loss_train: 6.544, loss_val: nan, pos_over_neg: 95.98627471923828 lr: 0.0001
Iter: 1405/3021, loss_train: 6.5496, loss_val: nan, pos_over_neg: 116.02179718017578 lr: 0.0001
Iter: 1406/3021, loss_train: 6.5585, loss_val: 

## Use sklearn to train classifier

In [ ]:
transforms_validation = torch.nn.Sequential(
    augmentation.ScaleDynamicRange(scaler_bounds=(0,1)),
    torchvision.transforms.Resize(size=(224,224),
                                  interpolation=torchvision.transforms.InterpolationMode.BILINEAR), 
    augmentation.TileChannels(dim=0, n_channels=3),
#     augmentation.Normalize(  means=[0.485, 0.456, 0.406],
#                              stds=[0.229, 0.224, 0.225]),
#     torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225],
#                                      inplace=False),
)
scripted_transforms_validation = torch.jit.script(transforms_validation)
# scripted_transforms = transforms


In [ ]:
dataset_labeled_train = util.dataset_simCLR(
                                    # torch.as_tensor(X_labeled_train, device='cpu', dtype=torch.float32), 
                                    torch.as_tensor(X_labeled_train_SYT, device='cpu', dtype=torch.float32), 
                                    # torch.as_tensor(torch.zeros(X_labeled_train.shape[0]), device='cpu', dtype=torch.float32),
                                    torch.as_tensor(torch.zeros(X_labeled_train_SYT.shape[0]), device='cpu', dtype=torch.float32),

                                    n_transforms=1,
                                    class_weights=np.array([1]),
                                    # class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
                                    transform=scripted_transforms_validation,
                                    # DEVICE='cpu',
                                    DEVICE='cpu',
                                    dtype_X=torch.float32,
                                    dtype_y=torch.int64,
                                    
                                    temp_uncertainty=16
                                    )
dataset_labeled_val = util.dataset_simCLR(
                                    # torch.as_tensor(X_labeled_val, device='cpu', dtype=torch.float32), 
                                    torch.as_tensor(X_labeled_val_SYT, device='cpu', dtype=torch.float32), 
                                    # torch.as_tensor(torch.zeros(X_labeled_val.shape[0]), device='cpu', dtype=torch.float32),
                                    torch.as_tensor(torch.zeros(X_labeled_val_SYT.shape[0]), device='cpu', dtype=torch.float32),

                                    n_transforms=1,
                                    class_weights=np.array([1]),
                                    # class_weights=np.array([1]*4)[np.random.randint(0,4, X_train.shape[0])],
                                    transform=scripted_transforms_validation,
                                    # DEVICE='cpu',
                                    DEVICE='cpu',
                                    dtype_X=torch.float32,
                                    dtype_y=torch.int64,
                                    
                                    temp_uncertainty=16
                                    )
dataloader_labeled_train = torch.utils.data.DataLoader( dataset_labeled_train,
    #                                                 batch_size=int(pct_batch_size*X_train.shape[0]),
                                                    batch_size=1024,
                                                    shuffle=False,
                                                    drop_last=False,
                                                    pin_memory=True,
                                                    num_workers=16,
                                                    persistent_workers=True,
                                                    # prefetch_factor=0
                                                    )
dataloader_labeled_val = torch.utils.data.DataLoader( dataset_labeled_val,
    #                                                 batch_size=int(pct_batch_size*X_train.shape[0]),
                                                    batch_size=1024,
                                                    shuffle=False,
                                                    drop_last=False,
                                                    pin_memory=True,
                                                    num_workers=16,
                                                    persistent_workers=True,
                                                    # prefetch_factor=0
                                                    )


In [ ]:
DEVICE = torch_helpers.set_device(use_GPU=True)

In [ ]:
model.eval()
model.to(DEVICE)
features_train = torch.cat([model.get_head(model.base_model(data[0][0].to(DEVICE))).detach().cpu() for data in dataloader_labeled_train], dim=0)
features_val   = torch.cat([model.get_head(model.base_model(data[0][0].to(DEVICE))).detach().cpu() for data in dataloader_labeled_val], dim=0)

In [ ]:
features_train.shape, features_val.shape

run a sweep of logistic regressions over C (1/L2) parameter

In [ ]:
acc_train, acc_val = [], []
# C_toUse = np.array([1000,100,10,1,0.1,0.01,0.001,0.0001,0.00001])
C_toUse = np.array([10,1,0.1,0.01,0.001,0.0001,0.00001])
for C in C_toUse:
#     print(f'C = {C}')
    logreg = sklearn.linear_model.LogisticRegression(solver='liblinear', max_iter=800, C=C)
#     tic = time.time()
    logreg.fit(features_train, y_labeled_train_SYT)
    # logreg.fit(features_train, y_labeled_train)
#     print(f'time: {time.time() - tic}')
    acc = logreg.score(features_train, y_labeled_train_SYT)
    # acc = logreg.score(features_train, y_labeled_train)
    acc_train.append(acc)
#     print(f'acc_train: {acc}')
    acc = logreg.score(features_val, y_labeled_val_SYT)
    # acc = logreg.score(features_val, y_labeled_val)
    acc_val.append(acc)
#     print(f'acc_val: {acc}')
#     print('')
    

plt.figure()
plt.plot(C_toUse, acc_train)
plt.plot(C_toUse, acc_val)
plt.xscale('log')
plt.xlabel('C')
plt.ylabel('acc')
plt.legend(['train', 'test']);

run a sinlg logistic regression with desired parameters and check confusion matrix

In [ ]:
logreg = sklearn.linear_model.LogisticRegression(solver='liblinear', max_iter=800, C=10**(1)).fit(features_train, y_labeled_train_SYT)

In [ ]:
%matplotlib inline
proba = logreg.predict_proba(features_train)

preds = np.argmax(proba, axis=1)
# cm = classification.confusion_matrix(preds, y_labeled_train)
cm = classification.confusion_matrix(preds, y_labeled_train_SYT)

plt.figure()
sns.heatmap(cm, annot=True, annot_kws={"size": 16}, vmax=1., cmap=plt.get_cmap('gray'))
plt.title('train');

proba = logreg.predict_proba(features_val)
preds = np.argmax(proba, axis=1)
# cm = classification.confusion_matrix(preds, y_labeled_val)
cm = classification.confusion_matrix(preds, y_labeled_val_SYT)

plt.figure()
sns.heatmap(cm, annot=True, annot_kws={"size": 16}, vmax=1., cmap=plt.get_cmap('gray'))
plt.title('val');


In [ ]:
list(model.state_dict().keys())

In [ ]:
model.state_dict()

In [ ]:
[print(key, '    ', layer.shape) for key, layer in model.state_dict().items()];

In [ ]:
layer_4.shape

In [ ]:
plt.figure()
plt.imshow(model.state_dict()['PostHead_0.weight'].cpu())

In [ ]:
%matplotlib notebook

layer_1 = model.state_dict()['base_model.0.0.0.weight'].cpu()
layer_2 = model.state_dict()['base_model.0.6.3.block.1.0.weight'].cpu()
layer_3 = model.state_dict()['base_model.0.7.0.block.1.0.weight'].cpu()
# layer_4 = model.state_dict()['base_model.0.7.0.block.1.0.weight'].cpu()


# plotting_helpers.plot_image_grid(torch.cat([arr for arr in layer_1], dim=0), grid_shape=(5,6), kwargs_imshow={'vmax': 0.2});
# plotting_helpers.plot_image_grid(torch.cat([arr for arr in layer_2], dim=0), grid_shape=(6,6), kwargs_imshow={'vmax': 0.2});
plotting_helpers.plot_image_grid(torch.cat([arr for arr in layer_3], dim=0), grid_shape=(4,6), kwargs_imshow={'vmax': 0.2});
